In [1]:
import sys
sys.path.append('../')

In [2]:
import numpy as np, pandas as pd
from glob import glob
import shutil, os
import pickle
import matplotlib.pyplot as plt
import torch.nn.functional as F
from sklearn.model_selection import GroupKFold
from sklearn.decomposition import IncrementalPCA
from tqdm.notebook import tqdm
import seaborn as sns
import PIL.Image as Image
import torch
from torch.utils.data import Dataset, DataLoader
import cv2
import gc
import time
import pandas_profiling as pdp
from pathlib import Path
from sklearn.model_selection import StratifiedKFold
from utils.util import *
from utils.losses import *
import torch.nn as nn
import transformers as T
import albumentations
import pandas as pd
import cv2
import numpy as np
import timm
import torch.nn as nn
from sklearn import metrics
import torch
from tez.callbacks import EarlyStopping
from tqdm import tqdm
import math
import tez
import torch.optim as optim
import warnings
warnings.simplefilter('ignore')

In [3]:
class CONFIG:
    DATA_PATH = Path('../input/petfinder-pawpularity-score')
    OUTPUT_DIR = Path('../output/5')
    MODEL_DIR = Path('../models/swin_large_patch4_window12_384')
    MODEL_NAME = 'swin_large_patch4_window12_384'
    batch_size = 8
    image_size = 384
    fold = 5
    epoch = 20
    lr = 2e-4
    device='cuda'
    training_step=True
    pretrained=False
    SEED=42
if not os.path.isdir(CONFIG.OUTPUT_DIR):
    os.makedirs(CONFIG.OUTPUT_DIR)
LOGGER = init_logger(OUTPUT_DIR=CONFIG.OUTPUT_DIR)
fix_seed(CONFIG.SEED)

In [4]:
train_df = pd.read_csv(CONFIG.DATA_PATH / 'train.csv')
train_df['path'] = train_df['Id'].map(lambda x:str(CONFIG.DATA_PATH/'train'/x)+'.jpg')
train_df['image_size'] = train_df['path'].apply(lambda image_id : Image.open(image_id).size)
train_df['width'] = train_df['image_size'].apply(lambda x: x[0])
train_df['height'] = train_df['image_size'].apply(lambda x: x[1])

test_df = pd.read_csv(CONFIG.DATA_PATH / 'test.csv')
test_df['path'] = test_df['Id'].map(lambda x:str(CONFIG.DATA_PATH/'test'/x)+'.jpg')
test_df['image_size'] = test_df['path'].apply(lambda image_id : Image.open(image_id).size)
test_df['width'] = test_df['image_size'].apply(lambda x: x[0])
test_df['height'] = test_df['image_size'].apply(lambda x: x[1])

train_df.head()

,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,Pawpularity,path,image_size,width,height
0,0007de18844b0dbbb5e1f607da0606e0,0,1,1,1,0,0,1,0,0,0,0,0,63,../input/petfinder-pawpularity-score/train/000...,"(405, 720)",405,720
1,0009c66b9439883ba2750fb825e1d7db,0,1,1,0,0,0,0,0,0,0,0,0,42,../input/petfinder-pawpularity-score/train/000...,"(1032, 774)",1032,774
2,0013fd999caf9a3efe1352ca1b0d937e,0,1,1,1,0,0,0,0,1,1,0,0,28,../input/petfinder-pawpularity-score/train/001...,"(720, 960)",720,960
3,0018df346ac9c1d8413cfcc888ca8246,0,1,1,1,0,0,0,0,0,0,0,0,15,../input/petfinder-pawpularity-score/train/001...,"(405, 720)",405,720
4,001dc955e10590d3ca4673f034feeef2,0,0,0,1,0,0,1,0,0,0,0,0,72,../input/petfinder-pawpularity-score/train/001...,"(540, 960)",540,960


In [5]:
train_df = get_train_data(train_df, train_df['Pawpularity'], n_splits = CONFIG.fold, regression=True)

In [6]:
train_aug = albumentations.Compose(
    [
        albumentations.Resize(CONFIG.image_size, CONFIG.image_size, p=1),
        albumentations.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
            max_pixel_value=255.0,
            p=1.0,
        ),
    ],
    p=1.0,
)

In [7]:
class PawpularDataset:
    def __init__(self, df, dense_features, targets, augmentations):
        self.image_paths = df['path'].tolist()
        self.dense_features = dense_features
        self.targets = targets
        if self.targets is None:
            self.targets = torch.ones(len(self.image_paths))
        self.augmentations = augmentations
        
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, item):
        image = cv2.imread(self.image_paths[item])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.augmentations is not None:
            augmented = self.augmentations(image=image)
            image = augmented["image"]
        targets = self.targets[item]
        image = np.transpose(image, (2, 0, 1)).astype(np.float32)
        
        features = self.dense_features[item, :]
        
        return {
            "image": torch.tensor(image, dtype=torch.float),
            "features": torch.tensor(features, dtype=torch.float),
            "targets": torch.tensor(targets, dtype=torch.float)
        }

In [15]:
class efficientnet_b0(nn.Module):
    def __init__(self,class_num):
        super().__init__()
        self.model = timm.create_model('efficientnet_b0', pretrained=True, in_chans=3)
        self.model.classifier = nn.Linear(self.model.classifier.in_features, 128)
        self.dropout = nn.Dropout(0.1)
        self.dense = nn.Linear(128
                               , class_num)

    def forward(self, image):
        x = self.model(image)
        x = self.dropout(x)
        x = self.dense(x)
        return x.squeeze(1)
class PawpularModel(nn.Module):
    def __init__(self, pet_classify_model, model_name):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=CONFIG.pretrained, in_chans=3)
        self.pet_classify_model = pet_classify_model
        self.model.head = nn.Linear(self.model.head.in_features, 128)
        self.dropout = nn.Dropout(0.1)
        self.dense1 = nn.Linear(140, 64)
        self.dense2 = nn.Linear(64, 1)

    def forward(self, image, features):
        x_ = self.model(image)
        x = self.dropout(x_)
        x = torch.cat([x, features], dim=1)
        x = self.dense1(x)
        x = self.dense2(x)
        p = F.softmax(self.pet_classify_model(F.adaptive_avg_pool2d(image, (224,224))), dim=1)
        return torch.cat([x, x_, features, p], axis=1)

In [16]:
def extraxt_feature(loader, extraxt_model, device):
    start = end = time.time()
    losses = AverageMeter()
    extraxt_model.eval()
    ex_feature = []
    for iter, data in tqdm(enumerate(loader), total=len(loader)):
        img, feature ,target = data['image'], data['features'], data['targets']
        img = img.to(device)
        feature = feature.to(device)
        target = target.to(device)
        with torch.no_grad():
            feature = extraxt_model(img, feature)
        ex_feature.append(feature)

    return torch.cat(ex_feature, axis=0)

In [17]:
def extract_loop(train, fold_):
    # ====================================================
    # Data Loader
    # ====================================================
    cl_model = efficientnet_b0(class_num=37)
    cl_model.to(CONFIG.device)
    cl_model.load_state_dict(fix_model_state_dict(torch.load('../input/The-Oxford-IIIT-Pet-Dataset/efficientnet_b0_Oxford_classifier_size_224.pth')["model"]))
    model = PawpularModel(pet_classify_model=cl_model, model_name=CONFIG.MODEL_NAME)
    model.to(CONFIG.device)
    cl_model=None
    gc.collect()
    if torch.cuda.device_count()>1:
        model=nn.DataParallel(model)
        model.load_state_dict(torch.load(CONFIG.MODEL_DIR / f"{CONFIG.MODEL_NAME}_{fold_}_best.pth")["model"],strict=False)
    else:
        model.load_state_dict(fix_model_state_dict(torch.load(CONFIG.MODEL_DIR / f"{CONFIG.MODEL_NAME}_{fold_}_best.pth")["model"]),strict=False)
    
    model.eval()    
    dense_features = [
        'Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory',
        'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur'
    ]
    pet_category = ['Abyssinian', 'Bengal', 'Birman', 'Bombay', 'British_Shorthair',
 'Egyptian_Mau' ,'Maine_Coon', 'Persian', 'Ragdoll', 'Russian_Blue' ,'Siamese',
 'Sphynx', 'american_bulldog' ,'american_pit_bull_terrier', 'basset_hound',
 'beagle', 'boxer' ,'chihuahua', 'english_cocker_spaniel', 'english_setter',
 'german_shorthaired' ,'great_pyrenees', 'havanese', 'japanese_chin',
 'keeshond', 'leonberger', 'miniature_pinscher', 'newfoundland', 'pomeranian',
 'pug' ,'saint_bernard' ,'samoyed' ,'scottish_terrier', 'shiba_inu',
 'staffordshire_bull_terrier' ,'wheaten_terrier' ,'yorkshire_terrier']
    
    val_idx = train[train.fold ==fold_].index
    valid_folds = train.loc[val_idx].reset_index(drop=True)
    
    val_dataset = PawpularDataset(
        valid_folds, 
        dense_features=valid_folds[dense_features].values, targets=valid_folds['Pawpularity']/100,
        augmentations=train_aug,
    )

    valid_loader = DataLoader(
        val_dataset,
        batch_size=CONFIG.batch_size,
        shuffle=False,
        pin_memory=True,
        drop_last=False,
    )
    
    # extract_feature
    feature = extraxt_feature(valid_loader, model,CONFIG.device)

    valid_folds[[f"embbeding_{CONFIG.MODEL_NAME}_{i}" for i in range(feature.shape[1])][:-len(pet_category)]+pet_category] = feature.data.cpu() 

    return valid_folds


In [18]:
def get_result(result_df):
    metric = RMSE()
    preds = result_df["Pawpularity"].values
    labels = result_df["preds"].values
    score = metric(preds, labels)
    LOGGER.info(f"Score: {score:<.5f}")

In [19]:
def inference():
    predictions = []
    dense_features = [
        'Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory',
        'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur'
    ]
    test_dataset = PawpularDataset(
        test_df, 
        dense_features=test_df[dense_features].values, targets=None,
        augmentations=train_aug,
    )
    test_loader = DataLoader(
        test_dataset, batch_size=16, shuffle=False, num_workers=4, pin_memory=True
    )
    features = []
    for fold_ in range(5):
        cl_model = efficientnet_b0(class_num=37)
        cl_model.to(CONFIG.device)
        cl_model.load_state_dict(fix_model_state_dict(torch.load('../input/The-Oxford-IIIT-Pet-Dataset/efficientnet_b0_Oxford_classifier_size_224.pth')["model"]))
        model = PawpularModel(pet_classify_model=cl_model, model_name=CONFIG.MODEL_NAME)
        model.to(CONFIG.device)
        cl_model=None
        gc.collect()
        if torch.cuda.device_count()>1:
            model=nn.DataParallel(model)
            model.load_state_dict(torch.load(CONFIG.MODEL_DIR / f"{CONFIG.MODEL_NAME}_{fold_}_best.pth")["model"],strict=False)
        else:
            model.load_state_dict(fix_model_state_dict(torch.load(CONFIG.MODEL_DIR / f"{CONFIG.MODEL_NAME}_{fold_}_best.pth")["model"]),strict=False)
        model.eval()
        ex_feature = []
        for iter, data in tqdm(enumerate(test_loader), total=len(test_loader)):
            img, feature ,target = data['image'], data['features'], data['targets']
            img = img.to(CONFIG.device)
            feature = feature.to(CONFIG.device)
            target = target.to(CONFIG.device)
            with torch.no_grad():
                feature = model(img, feature)
            ex_feature.append(feature.data.cpu())
        ex_feature = np.concatenate(ex_feature)
        features.append(ex_feature)
    feature = np.mean(np.stack(features, axis=0), axis=0)
    return feature

In [20]:
def main():
    # Training
    oof_df = pd.DataFrame()
    if CONFIG.training_step:
        for fold in range(CONFIG.fold):
            ex_ = extract_loop(train_df, fold)
            oof_df = pd.concat([oof_df, ex_])
        # Save OOF result
        oof_df.to_csv(CONFIG.OUTPUT_DIR / "oof_df.csv", index=False)
    else:
        oof_df = pd.read_csv(CONFIG.OUTPUT_DIR / "oof_df.csv")
    # CV result
    # Inference
    feature = inference()
    # submission
    pet_category = ['Abyssinian', 'Bengal', 'Birman', 'Bombay', 'British_Shorthair',
 'Egyptian_Mau' ,'Maine_Coon', 'Persian', 'Ragdoll', 'Russian_Blue' ,'Siamese',
 'Sphynx', 'american_bulldog' ,'american_pit_bull_terrier', 'basset_hound',
 'beagle', 'boxer' ,'chihuahua', 'english_cocker_spaniel', 'english_setter',
 'german_shorthaired' ,'great_pyrenees', 'havanese', 'japanese_chin',
 'keeshond', 'leonberger', 'miniature_pinscher', 'newfoundland', 'pomeranian',
 'pug' ,'saint_bernard' ,'samoyed' ,'scottish_terrier', 'shiba_inu',
 'staffordshire_bull_terrier' ,'wheaten_terrier' ,'yorkshire_terrier']
    test_feature = test_df.copy()
    test_feature[[f"embbeding_{CONFIG.MODEL_NAME}_{i}" for i in range(feature.shape[1])][:-len(pet_category)]+pet_category] = feature 
    return oof_df, test_feature

In [21]:
train_df, test_df = main()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.75it/s]


In [22]:
 pet_category = ['Abyssinian', 'Bengal', 'Birman', 'Bombay', 'British_Shorthair',
 'Egyptian_Mau' ,'Maine_Coon', 'Persian', 'Ragdoll', 'Russian_Blue' ,'Siamese',
 'Sphynx', 'american_bulldog' ,'american_pit_bull_terrier', 'basset_hound',
 'beagle', 'boxer' ,'chihuahua', 'english_cocker_spaniel', 'english_setter',
 'german_shorthaired' ,'great_pyrenees', 'havanese', 'japanese_chin',
 'keeshond', 'leonberger', 'miniature_pinscher', 'newfoundland', 'pomeranian',
 'pug' ,'saint_bernard' ,'samoyed' ,'scottish_terrier', 'shiba_inu',
 'staffordshire_bull_terrier' ,'wheaten_terrier' ,'yorkshire_terrier']

In [41]:
pd.set_option('max_columns', 300)
train_df

,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,Pawpularity,path,image_size,width,height,fold,embbeding_swin_large_patch4_window12_384_0,embbeding_swin_large_patch4_window12_384_1,embbeding_swin_large_patch4_window12_384_2,embbeding_swin_large_patch4_window12_384_3,embbeding_swin_large_patch4_window12_384_4,embbeding_swin_large_patch4_window12_384_5,embbeding_swin_large_patch4_window12_384_6,embbeding_swin_large_patch4_window12_384_7,embbeding_swin_large_patch4_window12_384_8,embbeding_swin_large_patch4_window12_384_9,embbeding_swin_large_patch4_window12_384_10,embbeding_swin_large_patch4_window12_384_11,embbeding_swin_large_patch4_window12_384_12,embbeding_swin_large_patch4_window12_384_13,embbeding_swin_large_patch4_window12_384_14,embbeding_swin_large_patch4_window12_384_15,embbeding_swin_large_patch4_window12_384_16,embbeding_swin_large_patch4_window12_384_17,embbeding_swin_large_patch4_window12_384_18,embbeding_swin_large_patch4_window12_384_19,embbeding_swin_large_patch4_window12_384_20,embbeding_swin_large_patch4_window12_384_21,embbeding_swin_large_patch4_window12_384_22,embbeding_swin_large_patch4_window12_384_23,embbeding_swin_large_patch4_window12_384_24,embbeding_swin_large_patch4_window12_384_25,embbeding_swin_large_patch4_window12_384_26,embbeding_swin_large_patch4_window12_384_27,embbeding_swin_large_patch4_window12_384_28,embbeding_swin_large_patch4_window12_384_29,embbeding_swin_large_patch4_window12_384_30,embbeding_swin_large_patch4_window12_384_31,embbeding_swin_large_patch4_window12_384_32,embbeding_swin_large_patch4_window12_384_33,embbeding_swin_large_patch4_window12_384_34,embbeding_swin_large_patch4_window12_384_35,embbeding_swin_large_patch4_window12_384_36,embbeding_swin_large_patch4_window12_384_37,embbeding_swin_large_patch4_window12_384_38,embbeding_swin_large_patch4_window12_384_39,embbeding_swin_large_patch4_window12_384_40,embbeding_swin_large_patch4_window12_384_41,embbeding_swin_large_patch4_window12_384_42,embbeding_swin_large_patch4_window12_384_43,embbeding_swin_large_patch4_window12_384_44,embbeding_swin_large_patch4_window12_384_45,embbeding_swin_large_patch4_window12_384_46,embbeding_swin_large_patch4_window12_384_47,embbeding_swin_large_patch4_window12_384_48,embbeding_swin_large_patch4_window12_384_49,embbeding_swin_large_patch4_window12_384_50,embbeding_swin_large_patch4_window12_384_51,embbeding_swin_large_patch4_window12_384_52,embbeding_swin_large_patch4_window12_384_53,embbeding_swin_large_patch4_window12_384_54,embbeding_swin_large_patch4_window12_384_55,embbeding_swin_large_patch4_window12_384_56,embbeding_swin_large_patch4_window12_384_57,embbeding_swin_large_patch4_window12_384_58,embbeding_swin_large_patch4_window12_384_59,embbeding_swin_large_patch4_window12_384_60,embbeding_swin_large_patch4_window12_384_61,embbeding_swin_large_patch4_window12_384_62,embbeding_swin_large_patch4_window12_384_63,embbeding_swin_large_patch4_window12_384_64,embbeding_swin_large_patch4_window12_384_65,embbeding_swin_large_patch4_window12_384_66,embbeding_swin_large_patch4_window12_384_67,embbeding_swin_large_patch4_window12_384_68,embbeding_swin_large_patch4_window12_384_69,embbeding_swin_large_patch4_window12_384_70,embbeding_swin_large_patch4_window12_384_71,embbeding_swin_large_patch4_window12_384_72,embbeding_swin_large_patch4_window12_384_73,embbeding_swin_large_patch4_window12_384_74,embbeding_swin_large_patch4_window12_384_75,embbeding_swin_large_patch4_window12_384_76,embbeding_swin_large_patch4_window12_384_77,embbeding_swin_large_patch4_window12_384_78,embbeding_swin_large_patch4_window12_384_79,embbeding_swin_large_patch4_window12_384_80,embbeding_swin_large_patch4_window12_384_81,embbeding_swin_large_patch4_window12_384_82,embbeding_swin_large_patch4_window12_384_83,embbeding_swin_large_patch4_window12_384_84,embbeding_swin_large_patch4_window12_384_85,embbeding_swin_large_patch4_window12_384_86,embbeding_swin_large_patch4_window12_384_87,em

In [53]:
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA
X_train, y_train = train_df.drop(columns=['Id', 'path', 'Pawpularity', 'image_size']), train_df['Pawpularity']
CV = []
ts = PCA(n_components=10)
def rmse(pred, target):
    pred = np.array(pred)
    target = np.array(target)
    score = np.sqrt((np.mean((pred - target)**2)))
    return score
for fold in range(5):
    X_tr, y_tr = X_train[X_train.fold!=fold].drop(columns=['fold']).values, y_train[X_train.fold!=fold].values
    X_val, y_val = X_train[X_train.fold==fold].drop(columns=['fold']).values, y_train[X_train.fold==fold].values
    regr = make_pipeline(ts,SVR(C=20))
    print(y_val)
    regr.fit(X_tr, y_tr)
    pred = regr.predict(X_val)
    print(pred)
    score = rmse(pred, y_val)
    print(score)
    CV.append(score)
print(f'CV::::{sum(CV)/len(CV)}')

[21 28 41 ... 18 20 30]
[32.24213514 32.15947636 34.36980533 ... 34.22171855 32.87590945
 33.93257232]
21.006474055343176
[34 27 36 ... 74 23 30]
[32.35368444 32.92063212 35.38912712 ... 34.50003825 33.00078873
 32.9291931 ]
20.89266347858259
[63 42 22 ... 31 15 70]
[32.93441989 34.84690428 35.03246971 ... 32.38146096 34.49468374
 34.92205111]
20.83517527558916
[28 15 74 ... 50 51 44]
[34.71366264 32.84117482 34.66562872 ... 32.13438061 34.56426205
 36.37956349]
20.899447553070125
[72 35  2 ... 44 31 20]
[34.21684293 32.24253524 33.28562864 ... 34.3070438  34.17798924
 34.2830595 ]
20.79043667496869
CV::::20.88483940751075


In [ ]:
a = torch.sigmoid(torch.tensor(train_df['embbeding_swin_large_patch4_window12_384_0'].values))*100
idx_0_10 = (0<train_df['Pawpularity']) & (train_df['Pawpularity']<10)y_val

In [36]:
import seaborn as sns

sns.distplot(
    a[], bins=100, color='#123456', label='data',
    kde=False,
    rug=False
)

SyntaxError: invalid syntax (847822051.py, line 4)

In [ ]:
sns.distplot(
    train_df['Pawpularity'], bins=100, color='#123456', label='data',
    kde=False,
    rug=False
)